<a href="https://colab.research.google.com/github/isao1974/ai_calc/blob/master/ai_calc_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備
csvファイルからデータを読み込み学習させる。

csvファイルはgdrive上に用意するためgdriveをマウントする。
colaboratory 上でのCSVファイルの読み込みは下記を参照
https://qiita.com/uni-3/items/201aaa2708260cc790b8

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

# 学習用データ作成

In [44]:
import pdb
pdb.set_trace()

f = random.random()

--Return--
> <ipython-input-44-7837b9b26f41>(2)<module>()->None
-> pdb.set_trace()
(Pdb) h

Documented commands (type help <topic>):
EOF    c          d        h         list      q        rv       undisplay
a      cl         debug    help      ll        quit     s        unt      
alias  clear      disable  ignore    longlist  r        source   until    
args   commands   display  interact  n         restart  step     up       
b      condition  down     j         next      return   tbreak   w        
break  cont       enable   jump      p         retval   u        whatis   
bt     continue   exit     l         pp        run      unalias  where    

Miscellaneous help topics:
exec  pdb

(Pdb) c
aaa


# 勉強用コード

学習済みモデルを使用する。
*   ph1: 加算する学習モデルを作成
*   ph2:四則演算する学習モデルを作成



https://qiita.com/Sa2Knight/items/221be46f2702ae453ba9
を参考に足し算を学習する学習モデルを作成する
モデルは保存する。

加算学習

In [0]:
from pandas import DataFrame
from sklearn import linear_model
import pickle

formulas = DataFrame([
    [0, 0],
    [0, 1],
    [0, 2],
    [1, 0],
    [1, 1],
    [1, 2],
    [2, 0],
    [2, 1],
    [2, 2]
])
answers = DataFrame([0, 1, 2, 1, 2, 3, 2, 3, 4])

model = linear_model.LinearRegression()
model.fit(formulas, answers)

with open('model.pickle', mode='wb') as fp:
  pickle.dump(model, fp)
  

while True:

     # 標準入力から計算式を取得
     print('> ', end='')
     inStr = input()
    
     if inStr == 'q':
      break
        
     x, y = list(map(lambda x: int(x), inStr.split(' ')))

     # 学習モデルを用いて回答を取得し、標準出力
     predected_answer = model.predict([[x, y]])
     print("{0} + {1} = {2}".format(x, y, int(predected_answer[0][0])))

保存したモデルを使う

DataFrame の１番目を演算子として学習モデルを作る。
linear_model.LinearRegression だと期待する結果が得られなかったので、
様々な識別器を試してみる。

識別器一覧
https://qiita.com/yhyhyhjp/items/3ca101f1cb5dee310c06

結果は失敗。
原因推測
* 学習データが少ない？
* モデルのハイパーパラメータが調整できてない？
* 機械学習では無理？ディープラーニングだといける？

In [0]:
from pandas import DataFrame
from sklearn import linear_model
import pickle

with open('model.pickle', mode='rb') as fp:
  model = pickle.load(fp)
  
  # 加算
  x, y = 10, 20
  predected_answer = model.predict([[x, y]])
  print("{0} + {1} = {3}".format(x, y, z, int(predected_answer[0])))

csv ファイルからデータ読み込み

In [0]:
import pandas as pd
from pandas import DataFrame
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split
import sys
import os
import pickle

h = .02  # step size in the mesh

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]


# test file existence
testDataFile = "drive/My Drive/Colab Notebooks/data.csv"
if os.path.isfile(testDataFile) == False:
  print("file not found")
  sys.exit()
  

# read file and split data
data = pd.read_csv(testDataFile, header=0)
X = data.loc[:, ['ope', '1st', '2nd']]
Y = data.loc[:, ['ans']]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)


#learning
for model in classifiers:
  print(model.__class__.__name__)  
  filename = "drive/My Drive/Colab Notebooks/"
  filename += model.__class__.__name__
  filename += ".pickle"
  print(filename)
  model.fit(X_train, Y_train)
  print(model.score(X_test, Y_test))
  print("-------")
  
  with open(filename, mode='wb') as fp:
    pickle.dump(model, fp)

学習モデルを使って推論
90%のスコアのSVCで推論させたが、数値が小さいときはいいけど、2桁になるともうダメ。

In [0]:
from pandas import DataFrame
from sklearn import linear_model
import pickle

filename = "drive/My Drive/Colab Notebooks/"
filename += "SVC"
filename += ".pickle"


with open(filename, mode='rb') as fp:
  model = pickle.load(fp)
  
  # 加算
  ope, x, y = 0, 100, 200
  predected_answer = model.predict([[ope, x, y]])
  print("{0} ,{1}, {2} = {3}".format(ope, x, y, int(predected_answer[0])))

deep learning で学習

In [39]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import plot_model
from sklearn.model_selection import train_test_split
import pandas as pd
import sys
import os

# test file existence
testDataFile = "drive/My Drive/Colab Notebooks/data.csv"
if os.path.isfile(testDataFile) == False:
  print("file not found")
  sys.exit()

# read file and split data
data = pd.read_csv(testDataFile, header=0)
X = data.loc[:, ['ope', '1st', '2nd']]
Y = data.loc[:, ['ans']]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)

# construct model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=3))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='linear'))
plot_model(model, to_file='DL_Model.png')
model.compile(loss='mean_squared_error', optimizer='Nadam', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=100, batch_size=64, verbose=0)
score = model.evaluate(X_test, Y_test)
print("loss=%f, acc=%f" % (score[0], score[1]))


#save model
import pickle
filename = "drive/My Drive/Colab Notebooks/"
filename += "DL_model"
filename += ".pickle"
print(filename)

with open(filename, mode='wb') as fp:
  pickle.dump(model, fp)

3816/3816 [==============================] - 1s 234us/step
loss=0.000014, acc=1.000000
drive/My Drive/Colab Notebooks/DL_model.pickle


deep learning の学習モデルを使って推論する

In [43]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import plot_model
from pandas import DataFrame
import pickle
import numpy as np

filename = "drive/My Drive/Colab Notebooks/"
filename += "DL_model"
filename += ".pickle"


with open(filename, mode='rb') as fp:
  model = pickle.load(fp)
  
  # 加算
  ope, x, y = 1, 10, 200
  predected_answer = model.predict(np.array([[ope, x, y]]))
  print("{0} , {1}, {2} = {3}".format(ope, x, y, int(predected_answer[0])))

1 , 10, 200 = 213
